# Finding the completeness function for all stars in my sample
I am going to try a somewhat more opaque machine-learning model to find the detection rate as a function of primary and secondary temperature, and primary and secondary vsini. We will probably have to treat the different instruments separately, but just in that there will be different best-fit parameters.

## Update:
The machine learning model works alright for predicting what kinds of companions you can detect for a new star. However, it does poorly in the interface region where the completeness drops from 1-->0. So, I ended up using 2D interpolation to get the completeness surfaces for each of my stars.

In [1]:
from __future__ import print_function, division, absolute_import

import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.cross_validation import cross_val_score
from matplotlib.cm import viridis
import sqlite3
from astropy.visualization import hist
import astropy.stats
from astropy.modeling import models, fitting
import seaborn as sns
import pickle
from scipy.interpolate import CloughTocher2DInterpolator
import warnings

sns.set_context('talk', font_scale=1.5)


%matplotlib inline

/home/kgullikson/anaconda3/envs/python2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Part 1: Machine Learning Model

In [ ]:
home = os.environ['HOME']
basedir = os.path.join(home, 'School', 'Research')

inst_dirs = dict(TS23=os.path.join(basedir, 'McDonaldData'),
                 HRS=os.path.join(basedir, 'HET_data'),
                 CHIRON=os.path.join(basedir, 'CHIRON_data'),
                 IGRINS=os.path.join(basedir, 'IGRINS_data'))

# Get the observing stats (especially the S/N)
obs_stats = pd.read_csv('data/SampleObservations.csv')

# Get the binarity/multiplicity of the stars
with sqlite3.connect('{}/.PythonModules/Stellar_database/Stars.sqlite'.format(os.environ['HOME'])) as db_con:
    sql_query = """
                   SELECT name, binarity.wds_binary, binarity.sb9_binary, binarity.my_binary, binarity.separation, binarity.K1 
                   FROM star
                   INNER JOIN binarity
                   ON star.id=binarity.star_id;
                """
    binarity = pd.read_sql_query(sql_query, db_con)
binarity['known_binary'] = ((binarity.wds_binary) | (binarity.sb9_binary) | (binarity.my_binary)).astype(bool)
binary_stars = binarity.loc[binarity.known_binary, 'name'].values

def get_summary(instrument, reject_binary=False):
    summary = pd.read_csv(os.path.join(inst_dirs[instrument], 'Sensitivity_Summary.csv'), index_col=0)
    if reject_binary:
        summary = summary.loc[~summary.star.isin(binary_stars)]
    ccf_data = pd.read_csv(os.path.join(inst_dirs[instrument], 'Cross_correlations', 'CCF_primary_20151129.rv.txt'))
    ccf_data.rename(columns=dict(teff='pri_teff', logg='pri_logg', vsini='pri_vsini'), inplace=True)
    tmp = pd.merge(summary, ccf_data[['star', 'date', 'pri_teff', 'pri_logg', 'pri_vsini']], on=['star', 'date'], how='left')
    return pd.merge(tmp, obs_stats.loc[obs_stats.instrument == instrument, ['star', 'date', 'exptime', 'snr']], 
                    on=('star', 'date'), how='left')
     
def get_injection_results(instrument, reject_binary=False):
    results = pd.read_csv(os.path.join(inst_dirs[instrument], 'Sensitivity_Dataframe.csv'), index_col=0)
    if reject_binary:
        results = results.loc[~results.star.isin(binary_stars)]
    ccf_data = pd.read_csv(os.path.join(inst_dirs[instrument], 'Cross_correlations', 'CCF_primary_20151129.rv.txt'))
    ccf_data.rename(columns=dict(teff='pri_teff', logg='pri_logg', vsini='pri_vsini'), inplace=True)
    tmp = pd.merge(results, ccf_data[['star', 'date', 'pri_teff', 'pri_logg', 'pri_vsini']], on=['star', 'date'], how='left')
    return pd.merge(tmp, obs_stats.loc[obs_stats.instrument == instrument, ['star', 'date', 'exptime', 'snr']], 
                    on=('star', 'date'), how='left')
    

In [ ]:
# Use CHIRON data for display and testing purposes (it has the most data)
df = get_injection_results('CHIRON', reject_binary=True)
summary = get_summary('CHIRON', reject_binary=True)
df['Detected'] = df.significance.notnull().astype(np.int)

## Find the importance of various features

In [ ]:
# Define features
features = ['pri_teff', 'temperature', 'pri_vsini', 'vsini', 'snr']
X = df.loc[df.addmode == 'simple', features].as_matrix()
y = df.loc[df.addmode == 'simple', 'Detected'].values

# Split into a training and test sample
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Fit the estimator
rf = RandomForestClassifier(n_estimators=50, max_depth=10)
rf.fit(X_train, y_train)

# Find the score on the test set
print('Classifier score = {:.2f}'.format(rf.score(X_test, y_test)))

# Show feature importance
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]
print('')
for f in range(X_train.shape[1]):
    print('{}: {} +/- {}'.format(features[indices[f]], importances[indices[f]], std[indices[f]]))
    
# Make a plot showing the feature importances
fig, ax = plt.subplots(1, 1, figsize=(14,7))
ax.bar(range(X_train.shape[1]), importances[indices], color='r', yerr=std[indices], align='center')
plt.xticks(range(X_train.shape[1]), [features[i] for i in indices])

## Do the same thing with a pipeline

I will actually use logistic regression instead of the random forest, but they are similar...

In [ ]:
cross_validate = True
classifier = 'SGD'

# Define features
features = ['temperature', 'pri_teff', 'vsini', 'pri_vsini', 'snr']
X = df.loc[df.addmode == 'simple', features].as_matrix()
y = df.loc[df.addmode == 'simple', 'Detected'].values

# Split into a training and test sample
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Fit the estimator
if classifier == 'SGD':
    clf = SGDClassifier(loss='log', alpha=0.001)
elif classifier == 'RF':
    clf = RandomForestClassifier(n_estimators=150, max_depth=10)
#clf = LogisticRegressionCV(Cs=20)
#clf = LinearSVC()

clf = CalibratedClassifierCV(clf, method='isotonic')
steps = [('Normalize', StandardScaler()), ('poly', PolynomialFeatures(2)), ('clf', clf)]
pipe = Pipeline(steps)

if cross_validate:
    # Cross-validate the SGD alpha parameter
    mean_score = []
    std_score = []
    if classifier == 'SGD':
        alphas = np.logspace(-6, 0, 10)
    elif classifier == 'RF':
        alphas = np.linspace(1, 300, 10, dtype=np.int)
    for alpha in alphas:
        print('alpha = {}'.format(alpha))
        if classifier == 'SGD':
            pipe.set_params(clf__base_estimator__alpha=alpha)
        else:
            pipe.set_params(clf__base_estimator__max_depth=alpha)
        scores = cross_val_score(pipe, X_train, y_train, cv=8, n_jobs=-1)
        mean_score.append(np.mean(scores))
        std_score.append(np.std(scores))

    plt.errorbar(alphas, mean_score, yerr=std_score)
    if classifier == 'SGD':
        ax = plt.gca()
        ax.set_xscale('log')

    # Set to the best value
    best_alpha = alphas[np.argmax(mean_score)]
    print('Setting alpha = {}'.format(best_alpha))
    if classifier == 'SGD':
        pipe.set_params(clf__base_estimator__alpha=best_alpha)
    elif classifier == 'RF':
        pipe.set_params(clf__base_estimator__max_depth=best_alpha)
    
pipe.fit(X_train, y_train)

# Find the score on the test set
print('Classifier score = {:.2f}'.format(pipe.score(X_test, y_test)))

In [ ]:
#pipe.set_params(clf__base_estimator__alpha=0.01)
X_summary = summary.loc[summary.addmode == 'simple', features]
if hasattr(pipe, "predict_proba"):
    prediction = pipe.predict_proba(X_summary)[:, 1]
else:  # use decision function
    prediction = pipe.decision_function(X_summary)
    prediction = \
                (prediction - prediction.min()) / (prediction.max() - prediction.min())
        
summary.loc[summary.addmode == 'simple', 'Predicted_detrate'] = prediction
sns.regplot('detrate', 'Predicted_detrate', data=summary, scatter_kws=dict(alpha=0.03))

In [ ]:
prob_pos = pipe.predict_proba(X_test)[:, 1]
fraction_of_positives, mean_predicted_value = \
            calibration_curve(y_test, prob_pos, n_bins=10)
plt.plot(mean_predicted_value, fraction_of_positives)
plt.xlabel('Mean Predicted Value')
plt.ylabel('Fraction of Positives')
plt.plot((0, 1), (0, 1), 'k--')

In [ ]:
#Fit to the full dataset
pipe.fit(X, y)

# Apply the fit to some stars and plot the residuals

In [ ]:
def plot_residuals(star, date, clf, features, addmode='simple', refit=False, **kwargs):
    if refit:
        #clf = clf.copy()
        subset = df.loc[(df.star == star) & (df.date == date) & (df.addmode == addmode)].copy()
        X = subset[features].as_matrix()
        y = subset['Detected'].values
        clf.fit(X, y)
        
    # Get predictions and residuals
    subset = summary.loc[(summary.star == star) & (summary.date == date) & (summary.addmode == addmode)].copy()
    X_summary = subset[features].as_matrix()
    if hasattr(clf, "predict_proba"):
        prediction = clf.predict_proba(X_summary)[:, 1]
    else:  # use decision function
        prediction = clf.decision_function(X_summary)
        prediction = \
                    (prediction - prediction.min()) / (prediction.max() - prediction.min())
    subset['prediction'] = prediction
    subset['diff'] = subset.detrate - subset.prediction
    
    # make figure
    fig = plt.figure(figsize=(14, 10))
    left = plt.subplot2grid((3, 2), (0, 0), rowspan=2)
    right = plt.subplot2grid((3, 2), (0, 1), rowspan=2)
    bottom = plt.subplot2grid((3, 2), (2, 0), colspan=2)
    
    # Plot 2d sensitivity curves
    #Sensitivity.heatmap(subset[['temperature', 'vsini', 'detrate']], ax=left, **kwargs)
    #Sensitivity.heatmap(subset[['temperature', 'vsini', 'diff']], ax=right, **kwargs)
    sns.heatmap(subset.pivot('temperature', 'vsini', 'detrate'), ax=left, annot=True, **kwargs)
    sns.heatmap(subset.pivot('temperature', 'vsini', 'diff'), ax=right, annot=True, **kwargs)
    left.set_title('Actual Sensitivity')
    right.set_title('Residuals')
    
    # Plot residual histogram
    residuals = subset.loc[subset['diff'].notnull(), 'diff'].values
    hist(residuals, bins=30)
    #bottom.set_yscale('log')
    #ylim = bottom.get_ylim()
    #bottom.set_ylim((0.5, ylim[1]))
    bottom.set_xlabel('Detection Rate Residuals')
    bottom.set_ylabel('Number')
    return 

star = 'HIP 93580'
date = summary.loc[summary.star == star].date.unique()[0]
residuals = plot_residuals(star, date, pipe, features, cmap=viridis, refit=False)


## Do the completeness curves look reasonable?

In [ ]:
def get_detrate(T2_vals, T1=10000, v1sini=100, v2sini=10.0, snr=500):
    T1_vals = np.ones_like(T2_vals) * T1
    v1sini_vals = np.ones_like(T2_vals) * v1sini
    v2sini_vals = np.ones_like(T2_vals) * v2sini
    snr_vals = np.ones_like(T2_vals) * snr
    X_tmp = np.vstack((T2_vals, T1_vals, v2sini_vals, v1sini_vals, snr_vals)).T
    y_tmp = pipe.predict_proba(X_tmp)[:, 1]
    return y_tmp

T2_vals = np.arange(3000, 12000, 10)
y1 = get_detrate(T2_vals, v2sini=0)
y2 = get_detrate(T2_vals, v2sini=50)
fig, ax = plt.subplots(1, 1, figsize=(14,7))
plt.plot(T2_vals, y1, label='S/N = 500')
plt.plot(T2_vals, y2, label='S/N = 100')
plt.legend(loc='best', fancybox=True)
plt.xlabel('Secondary Temperature')
plt.ylabel('Predicted Detection Rate')

# Conclusions:
It definitely gets the shape of the curve right, and reacts to all the parameters in the right way. The only issue is doesn't too very well in the transition region from detection rate = 1 --> 0. Moving on to...

# 2D Interpolation

In [ ]:
@models.custom_model
def sigmoid(x, alpha=-0.2, beta=20):
    f = alpha*(x-beta)
    return 1.0/(1+np.exp(-f))

def fit_sigmoid(x, y):
    m_init = sigmoid()
    f = fitting.LevMarLSQFitter()
    with warnings.catch_warnings():
        warnings.simplefilter('error')
        m = f(m_init, x, y, maxiter=10000)
    return m

#from itertools import cycle
#colors = cycle(('red', 'blue', 'green', 'black'))
def get_data(star, date, addmode='simple'):
    subset = summary.loc[(summary.star == star) & 
                         (summary.date == date) & 
                         (summary.addmode == addmode)].dropna(subset=('detrate',))
    
    # Add some points at low vsini and high temperature (always detected)
    highT = np.arange(7000, 12100, 200)
    tmp = pd.DataFrame(data=dict(temperature=highT, detrate=np.ones_like(highT)))
    for vsini in [0, 10, 20, 30, 40, 50]:
        idx = np.argmax(subset.loc[subset.vsini == vsini, 'temperature'])
        if subset.loc[idx, 'detrate'] == 1.0:
            tmp['vsini'] = vsini
            subset = pd.concat((subset, tmp))
    
    # Now, add some points at high vsini and low temperature (never detected)
    high_vsini = np.arange(75, 250+25, 25)
    tmp = pd.DataFrame(data=dict(vsini=high_vsini, detrate=np.zeros_like(high_vsini)))
    for teff in range(3000, 7000, 100):
        idx = np.argmax(subset.loc[subset.temperature == teff, 'vsini'])
        if subset.loc[idx, 'detrate'] == 0.0:
            tmp['temperature'] = teff
            tmp['detrate'] = 0.0
            subset = pd.concat((subset, tmp))
        elif subset.loc[idx, 'detrate'] < 0.5:
            vsini = subset.loc[subset.temperature == teff, 'vsini']
            detrate = subset.loc[subset.temperature == teff, 'detrate']
            try:
                sigmoid_fcn = fit_sigmoid(vsini.values, detrate.values)
            except:
                continue
            
            tmp['temperature'] = teff
            tmp['detrate'] = sigmoid_fcn(tmp['vsini'])
            subset = pd.concat((subset, tmp))
            #col = colors.next()
            #plt.scatter(vsini, detrate, c=col)
            #plt.plot(vsini, sigmoid_fcn(vsini), color=col)
        else:
            break
    return subset

In [ ]:
# Test out on a random star
star = summary.star.unique()[1]
date = summary.loc[summary.star == star, 'date'].unique()[0]
subset = get_data(star=star, date=date)

#subset = subset.loc[subset.temperature < 7000]
X = subset[['temperature', 'vsini']].as_matrix()
y = subset['detrate'].values

In [ ]:
xx = np.linspace(3000, 12000, num=200)
yy = np.linspace(0, 250, num=200)
XX, YY = np.meshgrid(xx, yy)
test_pts = np.array((XX, YY)).reshape(2, -1).T

# Interpolate
s = CloughTocher2DInterpolator(X, y, rescale=True)
mu = s(test_pts[:, 0], test_pts[:, 1])

# Plot
fig, (ax) = plt.subplots(1, 1, figsize=(14, 7))
im = ax.scatter(test_pts[:, 0], test_pts[:, 1], c=mu, s=50, marker='s', edgecolor='none', 
                        cmap=viridis, vmin=0, vmax=1)
im = ax.scatter(X[:, 0], X[:, 1], c=y, s=50, marker='s', cmap=viridis, vmin=0, vmax=1)
cax = plt.colorbar(im, ax=ax)
ax.set_xlim((2950, 12050))
ax.set_ylim((-3, 253))
ax.set_xlabel('Temperature (K)')
ax.set_ylabel('vsini (km/s)')
cax.set_label('Completeness')

In [ ]:
# Interpolate the completeness for every star.
output_dir = 'Completeness'
for star in summary.star.unique():
    for date in summary.loc[summary.star == star, 'date'].unique():
        # Get the data (supplemented with some assumption about the shape of the surface)
        subset = get_data(star=star, date=date)
        X = subset[['temperature', 'vsini']].as_matrix()
        y = subset['detrate'].values
        
        # Interpolate
        s = CloughTocher2DInterpolator(X, y, rescale=True)
        
        # Calculate the completeness on a large grid
        xx = np.linspace(3000, 12000, num=200)
        yy = np.linspace(0, 250, num=200)
        XX, YY = np.meshgrid(xx, yy)
        test_pts = np.array((XX, YY)).reshape(2, -1).T
        mu = s(test_pts[:, 0], test_pts[:, 1])

        # Plot
        fig, ax = plt.subplots(1, 1, figsize=(14, 7))
        im = ax.scatter(test_pts[:, 0], test_pts[:, 1], c=mu, s=50, marker='s', edgecolor='none', 
                                cmap=viridis, vmin=0, vmax=1)
        im = ax.scatter(X[:, 0], X[:, 1], c=y, s=50, marker='s', cmap=viridis, vmin=0, vmax=1)
        cax = plt.colorbar(im, ax=ax)
        ax.set_xlim((2950, 12050))
        ax.set_ylim((-3, 253))
        ax.set_xlabel('Temperature (K)')
        ax.set_ylabel('vsini (km/s)')
        cax.set_label('Completeness')
        
        # Save the interpolator and the figure
        fig.savefig('{}/{}_{}.pdf'.format(output_dir, 
                                          star.replace(' ', '_'), 
                                          date.replace('-', '')))
        plt.close('all')
        with open('{}/{}_{}.pkl'.format(output_dir, 
                                        star.replace(' ', '_'), 
                                        date.replace('-', '')), 'w') as outfile:
            pickle.dump(s, outfile)
        